# **Survey of Consumer Finances Analysis**
# Working Title: Perceptions and Reality of U.S. Consumer Finance Risks: An Analysis of COVID-19 effects
## Dan Valenzuela

***



## **Overview** <a id="Overview"></a>

[**1. Problem**](#Problem)

[**2. Data Understanding**](#Data-Understanding)

[**3. Data Preparation**](#Data-Preparation)

[**4. Data Analysis**](#Data-Analysis)

[**5. Evaluation**](#Evaluation)

[**6. Conclusion & Next Steps**](#Conclusion)

[**7. Endnotes**](#Endnotes)

***

## **Problem** <a id="Problem"></a>
[*↑ Back to overview*](#Overview)

WHat's the worse thing that could happen aside from household being decimated? Corona

[*↑ Back to overview*](#Overview)
***

## **Data Understanding** <a id="Data-Understanding"></a>
[*↑ Back to overview*](#Overview)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from CustomModules import dataloading as dl

targetdir = 'data/extracted/'

In [59]:
targetdir = 'data/extracted/'

df = dl.SCF_load_stata(targetdir, 2007, None)

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 22085 entries, 0 to 22084
Columns: 5831 entries, J7578 to J33001
dtypes: float32(263), float64(71), int16(2831), int32(372), int8(2294)
memory usage: 233.2 MB


In [45]:
targetdir = 'data/extracted/'

df = dl.CPS_raw(targetdir, ['jan2000'], None)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137006 entries, 0 to 137005
Columns: 360 entries, HRHHID to PWCMPWGT
dtypes: int64(360)
memory usage: 376.3 MB


In [46]:
df.head()

,HRHHID,HRMONTH,HRYEAR4,HURESPLI,HUFINAL,HUSPNISH,HETENURE,HEHOUSUT,HETELHHD,HETELAVL,...,PEGRPROF,PEGR6COR,PEMS123,PXDIPGED,PXHGCOMP,PXCYC,PXGRPROF,PXGR6COR,PXMS123,PWCMPWGT
0,5906350895161,1,2000,2,1,-1,2,1,1,-1,...,-1,-1,-1,1,1,1,1,1,1,19322613
1,5906350895161,1,2000,2,1,-1,2,1,1,-1,...,-1,-1,-1,50,1,0,1,1,1,21592067
2,5906350895161,1,2000,2,1,-1,2,1,1,-1,...,-1,-1,-1,1,1,1,1,1,1,27125991
3,8520510992294,1,2000,1,201,-1,1,1,1,1,...,-1,-1,-1,1,1,1,1,1,1,22011391
4,8520510992294,1,2000,1,201,-1,1,1,1,1,...,-1,-1,-1,1,1,0,1,1,1,19322613


In [47]:
dl.CPS_vars(targetdir, 'jan2000', None);

### Datasets

The data can be summarized by the [Federal Reserve](https://www.federalreserve.gov/publications/files/scf20.pdf) itself [<sup>1</sup>](#fn-1)<a id='fn-1-s'></a>: 

> The SCF is a triennial interview survey of U.S. families sponsored by the
Board of Governors of the Federal Reserve System with the cooperation of the U.S.
Department of the Treasury. Since 1992, data for the SCF have been collected by NORC, a
research organization at the University of Chicago. Although the majority of the data are
collected between May and December of each survey year, a small fraction of the data
collection occurs in the first four months of the next calendar year. In the 2019 SCF, this
portion of the data collection overlapped with early months of the COVID-19 pandemic,
with about 9 percent of interviews conducted between February and April 2020.

 

### Sampling
From [Federal Reserve Bulletin 2020 at 40](https://www.federalreserve.gov/publications/files/scf20.pdf)
>First, a standard multistage area-probability sample (a geographically based random sample) is selected
to provide good coverage of characteristics, such as homeownership, that are broadly
distributed in the population.
Second, a supplemental sample is selected to disproportionately include wealthy families,
which hold a relatively large share of such thinly held assets as noncorporate businesses
and tax-exempt bonds. Called the “list sample,” this group is drawn from a list of statistical
records derived from tax returns. These records are used under strict rules governing
confidentiality, the rights of potential respondents to refuse participation in the survey, and
the types of information that can be made available. Persons listed by Forbes as being
among the wealthiest 400 people in the United States are excluded from sampling.

>Of the 5,783 interviews completed for the 2019 SCF, 4,291 were from the area-probability
sample, and 1,492 were from the list sample; for 2016, 4,754 were from the area-probability
sample, and 1,500 were from the list sample.

### Weighting

From [Federal Reserve Bulletin 2020 at 42](https://www.federalreserve.gov/publications/files/scf20.pdf)

>To provide a measure of the frequency with which families similar to the sample families
could be expected to be found in the population of all families, an analysis weight is
computed for each case, accounting for both the systematic properties of the sample design
and differential patterns of nonresponse. The SCF response rates are low by the standards
of some other major government surveys, and analysis of the data confirms that the
tendency to refuse participation is highly correlated with net worth. However, unlike other
surveys, which almost certainly also have differential nonresponse by wealthy households,
the SCF has the means to adjust for such nonresponse.

From [Codebook 2019](https://www.federalreserve.gov/econres/files/codebk2019.txt)
> The issue of weighting in regressions has long been controversial.
Users of the SCF may find two references particularly useful:
(1) Analysis of Complex Surveys, C.J. Skinner, D. Holt, and
T.M.F. Smith (editors), John Wiley and Sons, 1989 (see particularly
pages 8-10, 154-157, and 286-287). (2) The Analysis of Household
Surveys: A Microeconometric Approach to Development Policy, Angus
Deaton, Johns Hopkins University Press, 1997 (see particularly pages
67-73).  

### Imputation

From [Codebook 2019](https://www.federalreserve.gov/econres/files/codebk2019.txt)

> The imputations are stored as five successive replicates
("implicates") of each data record.  Thus, the number of observations
in the full data set (28,915) is five times the actual number of
respondents (5,783)

>Users who want to estimate more complex statistics, particularly
regressions, should be cautious in their treatment of the implicates.
Some regression packages will treat each of the five implicates as an
independent observation and correspondingly inflate the reported
statistical significance of results.  Users who want to calculate
regression estimates, but who have no immediate use for proper significance
tests, could either average the dependent and independent values
across the implicates or multiply their standard errors by the square
root of five.  For an easily understandable discussion of multiple
imputation in the SCF from a user's point of view, see Catherine
Montalto and Jaimie Sung, "Multiple Imputation in the 1992 Survey of
Consumer Finances," Financial Counseling and Planning, Volume 7, 1996,
pages 133-146 (http://afcpe.org/assets/pdf/vol7_133-146multipleimputation.pdf).
That article also contains a set of simple SAS macros to
use to compute correct standard errors from multiply imputed data.
Two alternatives for processing general model estimates are offered
here, one written in SAS (MACRO MISECOMP) and the other in a Stata
ado file (micombine).  (NOTE: both SAS and Stata now include regression
packages for the analysis of multiply imputed data.)  See the section
"ANALYSIS WEIGHTS" below for a brief discussion of the inclusion of
sample design effects in the estimation of complex statistics

### Key Data for Merging Datasets and Analysis <a id="Key-Data" ></a>



[*↑ Back to overview*](#Overview)
***

## **Data Preparation** <a id="Data-Preparation"></a>
[*↑ Back to overview*](#Overview)

### Merging Data





### Cleaning Data <a id="Cleaning-Data"></a>



[*↑ Back to overview*](#Overview)
***

## **Data Analysis** <a id="Data-Analysis"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Evaluation**<a id="Evaluation"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Conclusion & Next Steps**<a id="Conclusion"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Endnotes** <a id="Endnotes"></a>
[*↑ Back to overview*](#Overview)


[1.](#fn-1-s)<a id='fn-1'></a> This is footnote 1

[*↑ Back to overview*](#Overview)